#### Dữ liệu Canada.xlsx chứa thông tin nhập cư vào Canada từ năm 1980 đến năm 2013. Bộ dữ liệu chứa dữ liệu hàng năm về dòng người di cư đến Canada được ghi nhận, trình bày thông tin inflows and outflows theo nơi sinh, quốc tịch hoặc nơi cư trú trước đó / tiếp theo cho cả người nước ngoài và quốc tịch. Chúng tôi sẽ tập trung vào dữ liệu nhập cư Canada.

In [1]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Đọc dữ liệu Canada.xlsx và lưu vào df_can, 
#    tìm hiểu về dữ liệu với: describe, head, shape, columns
df_can = pd.read_excel('data/Canada.xlsx', sheet_name='Canada by Citizenship', 
                        skiprows=range(20), skipfooter=2)

In [4]:
df_can.shape

(195, 43)

In [6]:
df_can.columns

Index([    'Type', 'Coverage',   'OdName',     'AREA', 'AreaName',      'REG',
        'RegName',      'DEV',  'DevName',       1980,       1981,       1982,
             1983,       1984,       1985,       1986,       1987,       1988,
             1989,       1990,       1991,       1992,       1993,       1994,
             1995,       1996,       1997,       1998,       1999,       2000,
             2001,       2002,       2003,       2004,       2005,       2006,
             2007,       2008,       2009,       2010,       2011,       2012,
             2013],
      dtype='object')

In [7]:
# df_can.info()

In [9]:
df_can.head()

,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Immigrants,Foreigners,Afghanistan,935,Asia,5501,Southern Asia,902,Developing regions,16,...,2978,3436,3009,2652,2111,1746,1758,2203,2635,2004
1,Immigrants,Foreigners,Albania,908,Europe,925,Southern Europe,901,Developed regions,1,...,1450,1223,856,702,560,716,561,539,620,603
2,Immigrants,Foreigners,Algeria,903,Africa,912,Northern Africa,902,Developing regions,80,...,3616,3626,4807,3623,4005,5393,4752,4325,3774,4331
3,Immigrants,Foreigners,American Samoa,909,Oceania,957,Polynesia,902,Developing regions,0,...,0,0,1,0,0,0,0,0,0,0
4,Immigrants,Foreigners,Andorra,908,Europe,925,Southern Europe,901,Developed regions,0,...,0,0,1,1,0,0,0,0,1,1


In [10]:
# Làm sạch dữ liệu:
# Bỏ đi những cột không cần thiết như 'AREA','REG','DEV','Type','Coverage' 
df_can.drop(['AREA','REG','DEV','Type','Coverage'], axis=1, inplace=True)

# Đổi tên một số cột như sau: 
#   'OdName' => 'Country', 'AreaName' => 'Continent', 'RegName' => 'Region'
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, 
              inplace=True)

# Thêm cột Total chứa tổng lượng nhập cư qua các năm
df_can['Total'] = df_can.sum(axis=1)

In [11]:
df_can.head()

,Country,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
0,Afghanistan,Asia,Southern Asia,Developing regions,16,39,39,47,71,340,...,3436,3009,2652,2111,1746,1758,2203,2635,2004,58639
1,Albania,Europe,Southern Europe,Developed regions,1,0,0,0,0,0,...,1223,856,702,560,716,561,539,620,603,15699
2,Algeria,Africa,Northern Africa,Developing regions,80,67,71,69,63,44,...,3626,4807,3623,4005,5393,4752,4325,3774,4331,69439
3,American Samoa,Oceania,Polynesia,Developing regions,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,6
4,Andorra,Europe,Southern Europe,Developed regions,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,15


In [12]:
# Tạo world map, với center [0, 0] là latitude và longitude, zoom level là 2, sử dụng tiles là OpenStreetMap
world_map = folium.Map(location=[0, 0], zoom_start=2, tiles='OpenStreetMap')
# world_map

In [13]:
# Tạo choropleth map sử dụng total nhập cư của từng quốc gia vào Canada từ năm 1980 đến năm 2013

# Lấy file GeoJSON có tên là world-countries.json
world_geo = r'data/world-countries.json'

folium.Choropleth(
    geo_data=world_geo,
    data=df_can,
    columns=['Country', 'Total'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Immigration to Canada'
).add_to(world_map)

world_map

In [15]:
df_can.sort_values(by='Total', ascending=False)[['Country','Total']].head()

,Country,Total
79,India,691904
36,China,659962
183,United Kingdom,551500
136,Philippines,511391
130,Pakistan,241600


In [16]:
df_can[df_can['Country']=='China']

,Country,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
36,China,Asia,Eastern Asia,Developing regions,5123,6682,3308,1863,1527,1816,...,42584,33518,27642,30037,29622,30391,28502,33024,34129,659962


In [17]:
df_can.loc[df_can['Country']=='China','Country']='Trung quoc'